In [1]:
import pandas as pd



In [20]:
# !pip install sklearn_crfsuite
import joblib
model = joblib.load("model/model.joblib")

In [21]:
stopwords = ["ผู้", "ที่", "ซึ่ง", "อัน"]

def tokens_to_features(tokens, i):
	word = tokens[i]

	features = {
		"bias": 1.0,
		"word.word": word,
		"word[:3]": word[:3],
		"word.isspace()": word.isspace(),
		"word.is_stopword()": word in stopwords,
		"word.isdigit()": word.isdigit(),
		"word.islen5": word.isdigit() and len(word) == 5
	}

	if i > 0:
		prevword = tokens[i - 1]
		features.update({
			"-1.word.prevword": prevword,
			"-1.word.isspace()": prevword.isspace(),
			"-1.word.is_stopword()": prevword in stopwords,
			"-1.word.isdigit()": prevword.isdigit(),
		})
	else:
		features["BOS"] = True

	if i < len(tokens) - 1:
		nextword = tokens[i + 1]
		features.update({
			"+1.word.nextword": nextword,
			"+1.word.isspace()": nextword.isspace(),
			"+1.word.is_stopword()": nextword in stopwords,
			"+1.word.isdigit()": nextword.isdigit(),
		})
	else:
		features["EOS"] = True

	return features

def parse(text):
	tokens = text.split()
	
	features = [tokens_to_features(tokens, i) for i in range(len(tokens))]

	output = model.predict([features])[0]
	return tokens, output

In [22]:
parse("นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330")

(['นายสมชาย',
  'เข็มกลัด',
  '254',
  'ถนน',
  'พญาไท',
  'แขวง',
  'วังใหม่',
  'เขต',
  'ปทุมวัน',
  'กรุงเทพมหานคร',
  '10330'],
 array(['O', 'O', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'LOC',
        'LOC', 'POST'], dtype=object))

In [23]:
parse("นายมงคล 123/4 ตำบล บ้านไกล อำเภอ เมือง จังหวัด ลพบุรี 15000")

(['นายมงคล',
  '123/4',
  'ตำบล',
  'บ้านไกล',
  'อำเภอ',
  'เมือง',
  'จังหวัด',
  'ลพบุรี',
  '15000'],
 array(['ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR',
        'POST'], dtype=object))

In [24]:
# text = "นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330"
text = '10330 นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร '
tokens = text.split()

In [25]:
features = [tokens_to_features(tokens, i) for i in range(len(tokens))]

In [26]:
feature_df = pd.DataFrame(features)
display(feature_df)

,bias,word.word,word[:3],word.isspace(),word.is_stopword(),word.isdigit(),word.islen5,BOS,+1.word.nextword,+1.word.isspace(),+1.word.is_stopword(),+1.word.isdigit(),-1.word.prevword,-1.word.isspace(),-1.word.is_stopword(),-1.word.isdigit(),EOS
0,1.0,นายสมชาย,นาย,False,False,False,False,True,เข็มกลัด,False,False,False,NaN,NaN,NaN,NaN,NaN
1,1.0,เข็มกลัด,เข็,False,False,False,False,NaN,254,False,False,True,นายสมชาย,False,False,False,NaN
2,1.0,254,254,False,False,True,False,NaN,ถนน,False,False,False,เข็มกลัด,False,False,False,NaN
3,1.0,ถนน,ถนน,False,False,False,False,NaN,พญาไท,False,False,False,254,False,False,True,NaN
4,1.0,พญาไท,พญา,False,False,False,False,NaN,แขวง,False,False,False,ถนน,False,False,False,NaN
5,1.0,แขวง,แขว,False,False,False,False,NaN,วังใหม่,False,False,False,พญาไท,False,False,False,NaN
6,1.0,วังใหม่,วัง,False,False,False,False,NaN,เขต,False,False,False,แขวง,False,False,False,NaN
7,1.0,เขต,เขต,False,False,False,False,NaN,ปทุมวัน,False,False,False,วังใหม่,False,False,False,NaN
8,1.0,ปทุมวัน,ปทุ,False,False,False,False,NaN,กรุงเทพมหานคร,False,False,False,เขต,False,False,False,NaN
9,1.0,กรุงเทพมหานคร,กรุ,False,False,False,False,NaN,10330,False,False,True,ปทุมวัน,False,False,False,NaN


# Streamlit

In [27]:
import streamlit as st


In [28]:
default_address_text = 'นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330'

In [29]:
st.title('Address Extraction')
address_text = st.text_input('Address', default_address_text)

if st.button('Submit'):
	tokens, output = parse(address_text)
	output_df = pd.DataFrame({
		'token': tokens,
		'output': output
	})
	st.write(output_df.T)


# Address Extraction

2024-11-07 19:14:45.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.530 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.532 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-07 19:14:45.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar